In [ ]:
import time
import re
import random
random.seed(2016)

import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import pipeline, grid_search
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer

In [ ]:
dtrain = xgb.DMatrix("train.buffer")
dtest = xgb.DMatrix("test.buffer")
evallist  = [(dtrain,'train')]

In [ ]:
loc = '%s'
#loc = '/home/ec2-user/data/hd/features/%s'
a_o = np.load(loc % 'train_data.npy')
b_o = np.load(loc % 'test_data.npy')
a_brand = np.load(loc % 'features_brand_01_train.npy')
b_brand = np.load(loc % 'features_brand_01_test.npy')
a_other = np.load(loc % 'FEATURES_1d_TRAIN.npy')
b_other = np.load(loc % 'FEATURES_1d_TEST.npy')
a_word_feat = np.load(loc % 'SPECIAL_WORDS_FEAT_TRAIN.npy')
b_word_feat = np.load(loc % 'SPECIAL_WORDS_FEAT_TEST.npy')

a_w2vdot = pd.read_pickle(loc % 'W2V_dots_train.df').drop('relevance', axis=1).values
b_w2vdot = pd.read_pickle(loc % 'W2V_dots_test.df').drop('relevance', axis=1).values

a_w2vdist = np.load(loc % 'W2V_dists_train.npz')['arr_0']
b_w2vdist = np.load(loc % 'W2V_dists_test.npz')['arr_0']

a_w2v_el = np.load(loc % 'W2V_vecs_train.npz')['arr_0']
b_w2v_el = np.load(loc % 'W2V_vecs_test.npz')['arr_0']

In [ ]:
# LOCAL CV
aaa = pd.read_pickle(loc % 'WOQTAL_TRAIN_147')
bbb = pd.read_pickle(loc % 'WOQTAL_TEST_147')
aa = aaa.drop('relevance', axis=1).values
bb = bbb.drop('relevance', axis=1).values
#aa = aaa.drop('relevance', axis=1).values #[:,[0,3]]
#bb = bbb.drop('relevance', axis=1).values #[:,[0,3]]

In [ ]:
#a = np.hstack((a_o, a_brand, a_other, a_word_feat, aa, a_w2vdot, a_w2vdist, a_w2v_el))
#b = np.hstack((b_o, b_brand, b_other, b_word_feat, bb, b_w2vdot, b_w2vdist, b_w2v_el))
a = np.hstack((a_o, a_brand, a_other, a_word_feat, aa, a_w2vdot, a_w2vdist, a_w2v_el[:,:700]))
b = np.hstack((b_o, b_brand, b_other, b_word_feat, bb, b_w2vdot, b_w2vdist, b_w2v_el[:,:700]))

In [ ]:
#FINAL GENERATE
aaa = pd.read_pickle(loc % 'WOQTAL_TRAIN_ALL')
bbb = pd.read_pickle(loc % 'WOQTAL_TEST_ALL')
aa = aaa.drop('relevance', axis=1).values
bb = bbb.drop('relevance', axis=1).values

a_full = np.hstack((a_o, a_brand, a_other, a_word_feat, aa, a_w2vdot, a_w2vdist, a_w2v_el))
b_full = np.hstack((b_o, b_brand, b_other, b_word_feat, bb, b_w2vdot, b_w2vdist, b_w2v_el))

### DATA SETS READY

In [ ]:
a.shape

In [ ]:
def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(a, dtrain.get_label(), test_size=0.20, random_state=147) # 0.20, 147
gX_train = xgb.DMatrix(data=X_train, label=y_train)
gX_test = xgb.DMatrix(data=X_test, label=y_test)
evallist  = [(gX_train,'train'),(gX_test,'test')]

In [ ]:
#clf = linear_model.LinearRegression(n_jobs=8)
#clf = linear_model.Lasso(alpha=0.01)
clf = linear_model.Ridge (alpha = 0.6)
clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)
y_hat = np.minimum(np.maximum(y_hat, 1.0), 3.0)
fmean_squared_error(y_hat, y_test)

### Checked LR

In [ ]:
def next_num_round():
    return np.random.randint(750, 950)

num_round = 5000

# TRAIN AS SELF
param = {'max_depth':9, 
         'eta':0.1, # 'objective':'reg:linear',
         'eval_metric':'rmse', #'maximize': False,
         'colsample_bytree':0.3,
         'subsample':0.9,
         'nthread':8,
         'silent': True
        }
# GOOD <<
param = {'max_depth':7, 
         'eta':0.03, # 'objective':'reg:linear',
         'eval_metric':'rmse', #'maximize': False,
         'colsample_bytree':0.7, #7
         'subsample':0.9,  #8
         'min_child_weight': 4.0,
         'nthread':32,
         'silent': True
        }
param = {'max_depth':9, 
     'eta':0.01, # 'objective':'reg:linear',
     'eval_metric':'rmse', #'maximize': False,
     'colsample_bytree':0.7, #7
     'subsample':0.9,  #8
     'min_child_weight': 4.0,
     'nthread':32,
     'silent': True
    }
bst = xgb.train( param, gX_train, num_round, [(gX_train,'train'),(gX_test,'test')], early_stopping_rounds=15, verbose_eval=10)
#bst = xgb.train( param, gX_train, num_round, [(gX_train,'train'),(gX_test,'test')], verbose_eval=10)

In [ ]:
# 700x trees  -  cv: 0.4377 (seed 147)
param = {'max_depth':9, 
         'eta':0.01, # 'objective':'reg:linear',
         'eval_metric':'rmse', #'maximize': False,
         'colsample_bytree':0.3, #0.8 #7  0.4449
         'subsample':0.8, #0.9,  #8
         'min_child_weight': 4.0,
         'nthread':32,
         'silent': True
        }  
num_round = 5000
bst = xgb.train( param, gX_train, num_round, [(gX_train,'train'),(gX_test,'test')], early_stopping_rounds=30, verbose_eval=10)

In [ ]:
ggX_train = xgb.DMatrix(data=a_full, label=dtrain.get_label())
ggX_test = xgb.DMatrix(data=b_full)

num_round = 1000
bst = xgb.train( param, ggX_train, num_round, [(gX_test,'test')], verbose_eval=50)

idx_train = pd.read_pickle(loc % 'LABELS_TRAIN.df')
idx_test = pd.read_pickle(loc % 'LABELS_TEST.df')

y_pred = bst.predict(ggX_test)
y_pred_bounded = np.minimum(np.maximum(y_pred, 1.0), 3.0)
idx_test['relevance'] = y_pred_bounded
idx_test.to_csv('submission_xgboost_words_0406_1720.csv')

In [ ]:
!gzip submission_xgboost_words_0405_1646.csv

In [ ]:
# 1500 trees 
param = {'max_depth':9, 
         'eta':0.01, # 'objective':'reg:linear',
         'eval_metric':'rmse', #'maximize': False,
         'colsample_bytree':0.8, #0.8 #7  0.4449
         'subsample':0.9, #0.9,  #8
         'min_child_weight': 4.0,
         'nthread':32,
         'silent': True
        }  
num_round = 5000
bst = xgb.train( param, gX_train, num_round, [(gX_train,'train'),(gX_test,'test')], early_stopping_rounds=15, verbose_eval=10)

with 0.30 and 1479
cv:  0.4493